In [4]:
from pys import credentials 

In [5]:
# Create Workspace

from azureml.core import Workspace

ws = Workspace.create(name                  = "handmade_ntt_data_challenge",
                      subscription_id       = credentials.azure_subscription_id,
                      resource_group        = credentials.azure_resource_group,
                      create_resource_group = False,
                      location              = credentials.azure_location)

Performing interactive authentication. Please follow the instructions on the terminal.


The default web browser has been opened at https://login.microsoftonline.com/organizations/oauth2/v2.0/authorize. Please continue the login in the web browser. If no web browser is available or if the web browser fails to open, use device code flow with `az login --use-device-code`.


Interactive authentication successfully completed.
Deploying StorageAccount with name handmadestorage822b6e018.
Deploying AppInsights with name handmadeinsights4f9e508a.
Deployed AppInsights with name handmadeinsights4f9e508a. Took 4.13 seconds.
Deploying KeyVault with name handmadekeyvault0f212c84.
Deployed KeyVault with name handmadekeyvault0f212c84. Took 18.55 seconds.
Deploying Workspace with name handmade_ntt_data_challenge.
Deployed StorageAccount with name handmadestorage822b6e018. Took 23.62 seconds.
Deployed Workspace with name handmade_ntt_data_challenge. Took 35.61 seconds.


In [6]:
# Connect to Workspace

from azureml.core import Workspace

ws = Workspace(workspace_name        = "handmade_ntt_data_challenge",
               subscription_id       = credentials.azure_subscription_id,
               resource_group        = credentials.azure_resource_group)

In [45]:
# Register the model

from azureml.core.model import Model

model = Model.register(workspace  = ws, 
                       model_path = "../resources/model.pkl", 
                       model_name = "handmade_model")

Registering model handmade_model


In [61]:
# Register the x_scaler

from azureml.core.model import Model

model = Model.register(workspace  = ws, 
                       model_path = "../resources/x_scaler.pkl", 
                       model_name = "handmade_x_scaler")

Registering model handmade_x_scaler


In [19]:
# Define inference config

from azureml.core import Environment
from azureml.core.model import InferenceConfig

env = Environment(name = "handmade_env")
dummy_inference_config = InferenceConfig(environment      = env,
                                         source_directory = "../pys/",
                                         entry_script     = "../pys/score.py")

In [7]:
# Local deployment

from azureml.core.webservice import LocalWebservice

deployment_config = LocalWebservice.deploy_configuration(port=6789)

In [77]:
service = Model.deploy(workspace         = ws,
                       name              = "handmade-predict",
                       models            = [model],
                       inference_config  = dummy_inference_config,
                       deployment_config = deployment_config,
                       overwrite         = True)

service.wait_for_deployment(show_output = True)

/var/folders/10/5nbnv2696vlgb605wf3g6wwh0000gn/T/ipykernel_2419/428816697.py:1: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(workspace         = ws,


NotImplementedError: Models must either be of type azureml.core.model.Model or a str path to a file or folder.

In [13]:
# Cloud deployment

from azureml.core import Environment
from azureml.core.model import InferenceConfig

env = Environment.from_conda_specification(name='cloudenv', file_path="../resources/conda.yaml")

inference_config = InferenceConfig(environment=env, source_directory='../pys/', entry_script='../pys/score.py')

In [14]:
from azureml.core.model import Model

model = Model(ws, "handmade_model")
scaler = Model(ws, "handmade_x_scaler")

service = Model.deploy(workspace         = ws,
                       name              = "handmade-predict",
                       models            = [model, scaler],
                       inference_config  = inference_config,
                       deployment_config = deployment_config,
                       overwrite         = True)

service.wait_for_deployment(show_output = True)

/var/folders/10/5nbnv2696vlgb605wf3g6wwh0000gn/T/ipykernel_29699/652748521.py:6: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(workspace         = ws,


Generating Docker build context.
2023/03/28 07:24:11 Downloading source code...
2023/03/28 07:24:12 Finished downloading source code
2023/03/28 07:24:12 Creating Docker network: acb_default_network, driver: 'bridge'
2023/03/28 07:24:12 Successfully set up Docker network: acb_default_network
2023/03/28 07:24:12 Setting up Docker configuration...
2023/03/28 07:24:13 Successfully set up Docker configuration
2023/03/28 07:24:13 Logging in to registry: e4e0e96ebb3f40cdb53da54690e88f94.azurecr.io
2023/03/28 07:24:14 Successfully logged into e4e0e96ebb3f40cdb53da54690e88f94.azurecr.io
2023/03/28 07:24:14 Executing step ID: acb_step_0. Timeout(sec): 5400, Working directory: '', Network: 'acb_default_network'
2023/03/28 07:24:14 Scanning for dependencies...
2023/03/28 07:24:14 Successfully scanned dependencies
2023/03/28 07:24:14 Launching container with name: acb_step_0
Sending build context to Docker daemon  71.68kB
Step 1/21 : FROM mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:20230120.

In [47]:
import requests
import json

uri = service.scoring_uri
requests.get("http://localhost:6789")
headers = {"Content-Type": "application/json"}

data = {
        "edad": 40.0,
        "trabajo": 40.0,
        "deuda": 40.0,
        "saldo": 40.0,
        "vivienda": 40.0,
        "prestamo": 40.0,
        "duracion": 40.0,
        "fecha_contacto": 40.0,
        "campaign": 40.0,
        "tiempo_transcurrido": 40.0,
        "contactos_anteriores": 40.0,
        "target": 40.0,
        "contactado": 40.0,
        "desconocido": 40.0,
        "fijo": 40.0,
        "movil": 40.0,
        "casado": 40.0,
        "divorciado": 40.0,
        "soltero": 40.0,
        "exito": 40.0,
        "nuevo_cliente": 40.0,
        "otro": 40.0,
        "sin_exito": 0,
        "primaria": 1,
        "secundaria/superiores": 0,
        "universitarios": 40.0
      }

data = json.dumps(data)
response = requests.post(uri, data=data, headers=headers)
print(response.json())

test is {'edad': 40.0, 'trabajo': 40.0, 'deuda': 40.0, 'saldo': 40.0, 'vivienda': 40.0, 'prestamo': 40.0, 'duracion': 40.0, 'fecha_contacto': 40.0, 'campaign': 40.0, 'tiempo_transcurrido': 40.0, 'contactos_anteriores': 40.0, 'target': 40.0, 'contactado': 40.0, 'desconocido': 40.0, 'fijo': 40.0, 'movil': 40.0, 'casado': 40.0, 'divorciado': 40.0, 'soltero': 40.0, 'exito': 40.0, 'nuevo_cliente': 40.0, 'otro': 40.0, 'sin_exito': 0, 'primaria': 1, 'secundaria/superiores': 0, 'universitarios': 40.0}
